In [1]:
# dataDB/chroma_scripts.ipynb  # load  < markdown + 图片  >
import os, hashlib, re, uuid 
from pathlib import Path
from typing import List, Union
from tqdm import tqdm

# 使用 Unstructured 更好地解析 Markdown 中的图片链接
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata # 过滤复杂元数据 chroma只支持: str int float bool None

In [2]:
markdown_file = r"..\docs.log\zhuanli_RobotHand\CN202021894937.5-一种结构紧凑的回转动力单元以及应用其的机器人\auto\CN202021894937.5-一种结构紧凑的回转动力单元以及应用其的机器人.md"
ims_dir = Path(markdown_file).parent / "images"
data_dir = os.path.dirname(markdown_file)
print(bool(Path(markdown_file).is_file())) # True 
print(bool(Path(ims_dir).is_dir()))  # True 

True
True


In [34]:
from IPython.display import display, Markdown, Image
# # 可视化 docs -> markdown
# display(Markdown(markdown_file))   # 图仅显示了文本部分，图片未显示

In [35]:
# data_dir/
#    + images/        # xx.jpg or xx.png ...
#    + markdown_file  # xxx.md

# docs = []
# loader = UnstructuredMarkdownLoader(markdown_file, mode="single") # single   elements
# # UnstructuredMarkdownLoader 底层使用 unstructured 库将 Markdown 转为 HTML，
# # 再用 lxml 提取文本，默认会过滤掉 <img> 标签，导致图片路径丢失

In [ ]:
from collections import OrderedDict
import os, re
from pathlib import Path
from pypdf import PdfReader
from langchain_core.documents import Document
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from typing import List 

class zhuanli_parser:
    def __init__(self, markdown_file: str):
        self.markdown_file = markdown_file
        self.ims_dir = Path(markdown_file).parent / "images"
        self.data_dir = os.path.dirname(markdown_file)
        self.loaded_docs: list[Document] = []
        self.meta_schema = OrderedDict({
            "pubno": str,
            "patent_name": str,
            "applier": str,
            "apply_time": str,
            "root_dir": Path(markdown_file).parent,
            "fig_list": dict,   # {"图1": ["描述", "绝对路径"]}
        }) # 申请公布号 专利名称 申请人 发明人 申请时间 配图{"图1": [str(description),str(path/to/1.jpg)], }, 路径
    
    def __call__(self):
        self.pipeline()  
        
    def _extract_imMatadata(self) -> dict:
        """  
        返回形如 {"fig_list": {"图1":[desc,abs_path], ...}}
        若无附图章节，返回 {"fig_list": {}}
        """
        content = Path(self.markdown_file).read_text(encoding='utf-8')

        """ 从markdown文本中提取图片元数据 """
        # 1 提取“附图说明”标题及其内容（直到下一个标题或文件结尾）
        pattern = re.compile(
            r'^(#{1,3})\s*附图说明\s*\n+\s*([\s\S]*?)(?=^#{1,3}|\Z)',
            re.MULTILINE)
        match = pattern.search(content)
        if not match:
            print(f"⚠️ markdown file {os.path.basename(self.markdown_file)} 未找到 '附图说明' 章节，跳过处理。")
            return {"fig_list": []}

        # header_line = match.group(1)  # 比如 "##"
        body = match.group(2).strip()   # 附图说明 段落 :  图x是xxxxx， ； 。
        img_map = {}
         # 2 抓所有 ![...](path) 图 1\n 图 2\n ...
        #    用两个捕获组：路径 和 图号
        img_blocks = re.findall(r'!\[.*?\]\((.*?)\)\s*\n\s*图(\d+)', content, re.IGNORECASE)
        for path, num in img_blocks:
            img_map[f"图{num}"] = path.strip()

        # 获取图片的图题： "图x": [decs, img_path]
        fig_num, desc = body.groups()   # 
        img_path = img_map.get(f"图{fig_num}")
        img_path = os.path.abspath(img_path)
        assert os.path.exists(img_path), f"图片路径不存在: {img_path}"
        if img_path:
            return {"图"+fig_num: [{desc}, str(img_path)]}
        else:
            print(f"⚠️ {os.path.basename(self.markdown_file)}未找到图{fig_num}的路径，跳过处理。")
            return match.group(0)  # 无图则保持原样
            
            
    
    def _extract_pubno(self):
        # 专利pdf第一页最后一行 -> 申请公告号
        pdfp = str(self.markdown_file)[:-3] + ".pdf"
        reader = PdfReader(pdfp)
        text_1 = reader.pages[0].extract_text() or ""
        last_line = text_1.strip().splitlines()[-1]

        # 去掉空格后匹配
        compact = re.sub(r'\s+', '', last_line.upper())
        m = re.search(r'(CN[A-Z0-9]{9,13})', compact)
        if m:
            return m.group(0)
        else:
            raise ValueError(f"专利 {os.path.basename(pdfp)} 未找到 申请公告号 的字符串")    
        
    
    def _load_markdown(self) -> List[Document]:
        # load markdown -> list[Document]
        loader = UnstructuredMarkdownLoader(str(self.markdown_file), mode="elements")
        
        
        
        return loader.load()
        
        
        # filter ends
        
        # add meta_data 
        
        # update self.loaded_docs
        return self.loaded_docs
        

    def pipeline(self):
        pass
    
zhuanli = zhuanli_parser(markdown_file=markdown_file)
imm = zhuanli._extract_imMatadata()
imm


AttributeError: 'str' object has no attribute 'groups'

In [ ]:
from pypdf import PdfReader

def extract_pubno(pdf_path: str) -> str | None:
    reader = PdfReader(pdf_path)
    text = reader.pages[0].extract_text() or ""
    last_line = text.strip().splitlines()[-1]

    # 去掉空格后匹配
    compact = re.sub(r'\s+', '', last_line.upper())
    m = re.search(r'(CN[A-Z0-9]{9,13})', compact)
    return m.group(0) if m else None

pdf_p = r"..\docs.log\zhuanli_RobotFeet\CN201721328994.5-一种机器人足端结构.pdf"
print(extract_pubno(pdf_p))  # CN207225508U

CN207225508U


In [ ]:
# 如果专利markdown中有完整的 附图说明  就直接重写这一节， 
# 没有 附图说明 的就跳过   --直接不要这个专利的图片信息
# 与之同步的， 给doc对象更新metadata

import re
from pathlib import Path
from typing import Dict
from langchain_community.document_loaders import PyPDFLoader
def extract_metadata(font_part_text: str) -> Dict:
    metadata_schema = {
        "info": {
            "申请公告号": "",   # 
            "专利名称": "",     # filename.stem
            "申请人": "",
            "申请时间": "",
        },
        "im_paths": {
            "图1-desc": "im_path_abs",
            },
    }
    
    return metadata_schema
    

def get_zhuanli_id() -> None:
    file_path = r"..\docs.log\zhuanli_RobotHand\CN202510118381.1-用于机器人灵巧手的直驱高转矩微特直流无刷空心杯电机.pdf"
    loader = PyPDFLoader(file_path)  
    data = loader.load()
    content = data[0].page_content
    


def rewrite_figure_section(md_text: str) -> str:
    """
    在原 markdown 中找到 # 附图说明 这一节，
    把 “图X 是 ……” 替换为
    “图X 是 ……，如图X 所示（![](images/xxx.jpg)）”
    其余内容不变。
    """
    # 1. 先把整段 # 附图说明 抠出来
    pattern_section = re.compile(
        r'(附图说明.*?\n)(.*?)(?=\n# |\n## |\Z)',
        re.DOTALL
    )
    m = pattern_section.search(md_text)
    if not m:
        return md_text

    section_start, section_body = m.group(1), m.group(2)

    # 2. 提取末尾所有图→路径映射
    img_map = {
        f"图{num}": path
        for path, num in re.findall(r'!\[.*?\]\((.*?)\)\s*\n图(\d+)', md_text)
    }

    # 3. 替换 section_body 中的 “图X 是 ……”
    def repl(match):
        fig, desc = match.groups()
        img_path = img_map.get(f"图{fig}")
        if img_path:
            return f"图{fig} 是{desc}，如图{fig}所示（![]({img_path})）"
        return match.group(0)

    new_body = re.sub(
        r'图(\d+)\s*是([^；;。]*?)[；;。]',
        repl,
        section_body
    )
    print(new_body)
    # # 4. 拼回去
    # new_section = section_start + new_body
    # return md_text.replace(m.group(0), new_section)

md_path = Path(markdown_file)
md_text = md_path.read_text(encoding="utf-8")
new_text = rewrite_figure_section(md_text)  # rewrite 没有必要



[0050] 图1 是本实用新型的爆炸图，如图1所示（![](images/a49f3951ef0a44229ff176558e4eeb418cea52b31045326a6cff3085b20a4309.jpg)）

[0051] 图2 是本实用新型的侧面示图，如图2所示（![](images/12a69fcf992d6c2f82c9ca90dff4cc2f021c5c815fc3b32d4a7e71ae4b757a8c.jpg)）  
[0052] 图3 是沿图2所示结构A-A剖视图，如图3所示（![](images/3ec15aeccd2870bc5cc9d22dffeef472674c8b13b19b6c89948b2f6970570c6f.jpg)）  
[0053] 图4 是本实用新型的柔性减速器的啮合示意图，如图4所示（![](images/53fb0eacf5fb283d05a7d91eace1edcf2f584a8e3f5c5d28b76fb303a49d5580.jpg)）  
[0054] 图5 是本实用新型的柔性减速器的内齿圈变形示意图，如图5所示（![](images/f1db420ca7d3ed295cab88fc8b8400bc74a30fab344bf1220355abab84fb2da6.jpg)）  
[0055] 图6 是本实用新型一种结构示图以及I处凹陷部结构的放大示图，如图6所示（![](images/80709eeee57f19f1d7c7e4f04ee4a7ed3b7ffc70dce2e676d311d86f56c9c305.jpg)）  
[0056] 图7 是本实用新型一种结构示图以及II处凸起部结构的放大示图，如图7所示（![](images/f3f442187a07542698e68681d6ba0e7bee18473e908ea37172a9716f799ab39c.jpg)）  
[0057] 图中：1、驱动板；2、转子编码器芯片；3、转子磁环；4、支撑杆；41、输出端编码器芯片;5、电机转子;6、输出端磁环；7、行星架;8、柔性内齿圈；12、行星轮;13、太阳轮；17、基座；  
18、凸起部；19、凹陷部；A、原轮廓；B、变形后轮廓。具体实施方式  
[0058]为了使本实用新型的目的、技术方案及优点更

✅ 模型加载完成: model_name='../deepdocs/mineru_models/Qwen3-Embedding-0.6B' cache_folder=None model_kwargs={'trust_remote_code': True} encode_kwargs={'normalize_embeddings': True} query_encode_kwargs={} multi_process=False show_progress=False


In [ ]:
import re
from pathlib import Path
from typing import List, Dict, Tuple 

def get_imsMatadata(md_path: Path) -> Dict[str, str]:
    text = md_path.read_text(encoding='utf-8')
    root_dir = md_path.parent
    img_map = {}
    # 匹配：图片行 + 紧接着的“图X”行
    img_blocks = re.findall(r'!\[.*?\]\((.*?)\)\s*\n\s*图(\d+)', text, re.IGNORECASE)
    for path, num in img_blocks:
        img_map[f"图{num}"] = path.strip()
    return img_map


def delete_appendix(langchain_md_text: str) -> str:
    """ 删除文末所有“图片 + 图X”块（连续出现在末尾的） """
    new_text = re.sub(
        r'(\n\s*!\[.*?\]\([^)]+\)\s*\n\s*图\d+\s*)+$',
        '',
        langchain_md_text,
        flags=re.MULTILINE | re.IGNORECASE
    ).rstrip() + '\n'
    return new_text 


def refine_patent_markdown(md_path: Path) -> str:
    text = md_path.read_text(encoding='utf-8')
    
    root_dir = md_path.parent

    # 1️⃣ 提取“附图说明”标题及其内容（直到下一个标题或文件结尾）
    pattern = re.compile(
        r'^(#{1,3})\s*附图说明\s*\n+\s*([\s\S]*?)(?=^#{1,3}|\Z)',
        re.MULTILINE)
    match = pattern.search(text)
    if not match:
        print(f"⚠️ markdown file {md_path.stem} 未找到 '附图说明' 章节，跳过处理。")
        return text

    header_line = match.group(1)  # 比如 "##"
    body = match.group(2).strip()

    # 2️⃣ 提取文末：![](path.jpg) 后接 “图1”、“图2” 的映射
    # 支持格式：
    #   ![图1](fig1.jpg)
    #   图1
    # 或者：
    #   ![](fig2.jpg)
    #   图2
    img_map = {}
    # 匹配：图片行 + 紧接着的“图X”行
    img_blocks = re.findall(r'!\[.*?\]\((.*?)\)\s*\n\s*图(\d+)', text, re.IGNORECASE)
    for path, num in img_blocks:
        img_map[f"图{num}"] = path.strip()

    # 3️⃣ 增强“图X 是...”这类句子，插入图片引用
    def repl(match):
        fig_num, desc = body.groups()
        img_path = img_map.get(f"图{fig_num}")
        if img_path:
            # 保留原句标点（使用 match.group() 获取完整匹配，再手动处理）
            full_match = match.group(0)
            # 去掉末尾标点，统一处理
            punct = full_match[-1]
            # return f"图{fig_num}是{desc}，如图{fig_num}所示（![]({img_path})）{punct}"
                
        else:
            return match.group(0)  # 无图则保持原样

    # 匹配：图1 是……。图2 是……；
    enhanced_body = re.sub(
        r'图(\d+)\s*是([^；;。]*?)[；;。]',
        repl,
        body
    )
    print(enhanced_body)
    # 4️⃣ 构造新的“附图说明”章节
    new_section = f"{header_line} 附图说明\n\n{enhanced_body}"

    # 5️⃣ 替换原文中的旧章节
    new_text = text[:match.start()] + new_section + text[match.end():]

    # 6️⃣ 删除文末所有“图片 + 图X”块（连续出现在末尾的）
    # 清理模式：任意数量的 \n + 图片行 + 图号行，重复出现，直到结尾
    new_text = re.sub(
        r'(\n\s*!\[.*?\]\([^)]+\)\s*\n\s*图\d+\s*)+$',
        '',
        new_text,
        flags=re.MULTILINE | re.IGNORECASE
    ).rstrip() + '\n'

    return new_text


# ✅ 使用示例
md_file = Path(markdown_file)    
new_text = refine_patent_markdown(md_file)

# 写回文件（可选）
# md_file.write_text(new_text, encoding='utf-8')
# print(new_text)

[0050] 图1是本实用新型的爆炸图，如图1所示（![](images/a49f3951ef0a44229ff176558e4eeb418cea52b31045326a6cff3085b20a4309.jpg)）；

[0051] 图2是本实用新型的侧面示图，如图2所示（![](images/12a69fcf992d6c2f82c9ca90dff4cc2f021c5c815fc3b32d4a7e71ae4b757a8c.jpg)）；  
[0052] 图3是沿图2所示结构A-A剖视图，如图3所示（![](images/3ec15aeccd2870bc5cc9d22dffeef472674c8b13b19b6c89948b2f6970570c6f.jpg)）；  
[0053] 图4是本实用新型的柔性减速器的啮合示意图，如图4所示（![](images/53fb0eacf5fb283d05a7d91eace1edcf2f584a8e3f5c5d28b76fb303a49d5580.jpg)）；  
[0054] 图5是本实用新型的柔性减速器的内齿圈变形示意图，如图5所示（![](images/f1db420ca7d3ed295cab88fc8b8400bc74a30fab344bf1220355abab84fb2da6.jpg)）；  
[0055] 图6是本实用新型一种结构示图以及I处凹陷部结构的放大示图，如图6所示（![](images/80709eeee57f19f1d7c7e4f04ee4a7ed3b7ffc70dce2e676d311d86f56c9c305.jpg)）；  
[0056] 图7是本实用新型一种结构示图以及II处凸起部结构的放大示图，如图7所示（![](images/f3f442187a07542698e68681d6ba0e7bee18473e908ea37172a9716f799ab39c.jpg)）。  
[0057] 图中：1、驱动板；2、转子编码器芯片；3、转子磁环；4、支撑杆；41、输出端编码器芯片;5、电机转子;6、输出端磁环；7、行星架;8、柔性内齿圈；12、行星轮;13、太阳轮；17、基座；  
18、凸起部；19、凹陷部；A、原轮廓；B、变形后轮廓。具体实施方式  
[0058]为了使本实用新型的目的、技术方案及优点更加

In [ ]:
# import re
# from pathlib import Path
# from typing import List
# from langchain_core.documents import Document
from typing import Dict

class markdown_loader:
    def __init__(self, markdown_file: str):
        self.markdown_file = markdown_file
        self.ims_dir = Path(markdown_file).parent / "images"
        self.data_dir = os.path.dirname(markdown_file)
        self.IMAGE_REGEX = re.compile(r'!\[.*?\]\((.*?)\)')
        self.loaded_docs = []
    
    def __call__(self):
        self.loaded_docs = self._load_with_image()
        
    def _extract_image_paths(self, content: str) -> List[str]:
        """从 Markdown 内容中提取图片路径"""
        return self.IMAGE_REGEX.findall(content)
    
    def _load_with_image(self) -> List[Document]:
        with open(self.markdown_file, "r", encoding='utf-8') as f:
            content = f.read()
            image_paths = self._extract_image_paths(content)
            abs_image_paths = [os.path.join(self.data_dir, image_path) for image_path in image_paths]
            
            doc = Document(
                page = content,
                metadata = {
                    "source_path": str(self.markdown_file),
                    "file_stem": os.path.basename(self.markdown_file).split(".")[0],
                    "file_suffix": os.path.basename(self.markdown_file).split(".")[1],
                    "file_size": os.path.getsize(self.markdown_file),
                    "image_paths": abs_image_paths
                }
            ) 
            return [doc]
        




    
    




In [6]:
for i, doc in enumerate(docs):
    display(Markdown(f"## 📄 Document {i}"))
    display(Markdown(doc.page_content))

    # 如果有图片路径，尝试显示
    if "image_paths" in doc.metadata:
        for img_path in doc.metadata["image_paths"]:
            if Path(img_path).exists():
                display(Image(filename=img_path))

In [ ]:
# embedding
# local_embedding_dir = r"../temp/mineru_models/Qwen3-Embedding-0.6B"
local_embedding_dir = r"../deepdocs/mineru_models/Qwen3-Embedding-0.6B" 
embedding_model = HuggingFaceEmbeddings(
    model_name=local_embedding_dir,
    model_kwargs={"trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True},
    cache_folder=None,  # 不使用额外缓存
)
print(f"✅ 模型加载完成: {embedding_model}")

✅ 模型加载完成: model_name='../deepdocs/mineru_models/Qwen3-Embedding-0.6B' cache_folder=None model_kwargs={'trust_remote_code': True} encode_kwargs={'normalize_embeddings': True} query_encode_kwargs={} multi_process=False show_progress=False
